In [2]:
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

In [6]:
def read_path(patient, type):
    data = []
    # Patient -> 0:BAK, 1:IM, 2:LEE
    # Type -> 0:ARIA, 1:AiCE, 2:IR
    root = "G:/Exported/"
    patients = ["BAK_MYEONGHYI", "IM_SUKYEONG", "LEE_HYEONJA"]
    recon_type = ["ARIA", "AiCE", "IR"]
    path = f"{root}{patients[patient]}/{recon_type[type]}/"
    
    for item in os.listdir(path):
        data.append(f"{path}{item}")

    return data

In [7]:
bak_aria = read_path(0, 0)
bak_aice = read_path(0, 1)
bak_ir = read_path(0, 2)
bak = [bak_aria, bak_aice, bak_ir]

im_aria = read_path(1, 0)
im_aice = read_path(1, 1)
im_ir = read_path(1, 2)
im = [im_aria, im_aice, im_ir]

lee_aria = read_path(2, 0)
lee_aice = read_path(2, 1)
lee_ir = read_path(2, 2)
lee = [lee_aria, lee_aice, lee_ir]

patients = [bak, im, lee]

In [8]:
def dice_custom(A, B):
    A_flat = A.reshape(A.size)
    B_flat = B.reshape(B.size)
    
    intersection = 0
    for i in range(B.size):
        if A_flat[i] == B_flat[i]:
            intersection += 1
    return (2*intersection)/(np.sum(A) + np.sum(B))

In [9]:
def dice_score(A, B):
    return (2*np.sum(A*B)/(np.sum(A)+np.sum(B)))

In [10]:
def compare_organs(patient):
    client = patients[patient]  # 어떤 환자의 데이터를 사용할 것인지
    aice = client[1]
    ir = client[2]
    names = ["BAK_MYEONGHYI", "IM_SUKYEONG", "LEE_HYEONJA"]

    dice_results = {}
    for i in aice:
        organ = i.split('/')[4].split('_')[3].split('.')[0]
        A = nib.load(i).get_fdata()
        for j in ir:
            if organ in j:
                B = nib.load(j).get_fdata()
        dice_results[organ] = dice_score(A, B)

    df = pd.DataFrame(list(dice_results.items()), columns=['Organ', 'Dice Coefficient'])
    df.to_excel(f"G:/Compare_CSV/{names[patient]}_AiCE_IR.xlsx")
    
    return

In [19]:
def compare_ctv(patient):
    client = patients[patient]
    if patient == 2:  # LEE -> CTV vs Brain
        for i in client[0]:
            if "CTV" in i:
                A = nib.load(i).get_fdata()
        for j in client[1]:
            if "Brain" in j:
                B = nib.load(j).get_fdata()
        for k in client[2]:
            if "Brain" in k:
                C = nib.load(k).get_fdata()
    else:  # BAK, IM -> CTV vs Breast
        for i in client[0]:
            if "CTV" in i:
                A = nib.load(i).get_fdata()
        for j in client[1]:
            if "Breast" in j:
                B = nib.load(j).get_fdata()
        for k in client[2]:
            if "Breast" in k:
                C = nib.load(k).get_fdata()
    
    ctv_aice = dice_score(A, B)
    ctv_ir = dice_score(A, C)
    
    return ctv_aice, ctv_ir

In [15]:
compare_organs(2)

KeyboardInterrupt: 

In [18]:
ctv_aice_bak, ctv_ir_bak = compare_ctv(0)
ctv_aice_im, ctv_ir_im = compare_ctv(1)
ctv_aice_lee, ctv_ir_lee = compare_ctv(2)

print(ctv_aice_bak, ctv_ir_bak)
print(ctv_aice_im, ctv_ir_im)
print(ctv_aice_lee, ctv_ir_lee)

ValueError: operands could not be broadcast together with shapes (512,512,92) (512,512,91) 

In [59]:
a=nib.load("G:/Exported/LEE_HYEONJA/AiCE/24910849_ST_23_Brain.nii.gz").get_fdata()
b=nib.load("G:/Exported/LEE_HYEONJA/IR/24910849_ST_24_Brain.nii.gz").get_fdata()
dice_score(a,b)

0.9960578730734511